# Pipeline


In [6]:
import pandas as pd
import requests
import numpy as np
import math
import pickle
import glob

In [7]:
def cambiar_minusculas_variable(archivo,variable):
    archivo[variable]=archivo[variable].str.lower()


In [ ]:

path = '../data' # use your path
all_files = glob.glob(path + "/*.zip")
all_files = all_files[0:3]

li = []
i = 0
for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    print(i)
    i+=1
    li.append(df)

data1 = pd.concat(li, axis=0, ignore_index=True)

0
1


In [ ]:

path = '../data' # use your path
all_files = glob.glob(path + "/*.zip")
all_files = all_files[3:6]

li = []
i = 0
for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    print(i)
    i+=1
    li.append(df)

data2 = pd.concat(li, axis=0, ignore_index=True)

In [ ]:

path = '../data' # use your path
all_files = glob.glob(path + "/*.zip")
all_files = all_files[6:9]

li = []
i = 0
for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    print(i)
    i+=1
    li.append(df)

data3 = pd.concat(li, axis=0, ignore_index=True)

In [ ]:

path = '../data' # use your path
all_files = glob.glob(path + "/*.zip")
all_files = all_files[9:12]

li = []
i = 0
for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    print(i)
    i+=1
    li.append(df)

data4 = pd.concat(li, axis=0, ignore_index=True)

In [ ]:

data = pd.concat([data1,data2,data3,data4], axis=0, ignore_index=True)

In [ ]:
#data = pd.read_csv('../data/201912-citibike-tripdata.csv.zip')

In [ ]:
#Creación de la columna trip_id

#generamos los ID's
n = len(data.index)
digits_range = range(1345,n + 1345)
digits_list = list(digits_range)

#Agreagamos la columna al df
data['trip_id'] = digits_list

In [ ]:
#Se obtienen las filas que tienen como usertype "Customer"
subset_c = data.loc[data['usertype'] == 'Customer']

#Se obtienen las filas que tienen como usertype "Subscriber"
subset_s = data.loc[data['usertype'] == 'Subscriber']

In [ ]:
subset_c.shape[0]

#Número de pases de 1 día
single_trip = round(subset_c.shape[0]*.684)

#Número de pases de 3 días
one_day = subset_c.shape[0] - single_trip

print("Numero de pases en diciembre de 1 día",one_day )
print("Numero de pases en diciembre de un viaje sencillo",single_trip )
one_day + single_trip

In [ ]:

#Creación de columna auxiliar para separar los pases de Costumer en 2
c_n = len(subset_c.index)
subset_c['aux']=np.random.binomial(1,.684,c_n)

In [ ]:
#Creación de columna con el nombre de los pases nuevos
subset_c['type_pass'] = np.where(subset_c['aux'] == 1, "single_trip", "one_day")
#Borramos la columa auxiliar
del subset_c['aux']

subset_s['type_pass']='annual'
#Juntamos los dos dataframes
data = subset_s.append(subset_c, ignore_index=True)
data.shape

In [ ]:
#Se obtienen las filas que tienen como usertype "Customer"
subset_s = data.loc[data['type_pass'] == 'single_trip']

subset_s['trip_category']='one-way'

subset_o = data.drop(subset_s.index)

In [ ]:
#Creación de una columna nueva : trip_category
o_n = len(subset_c.index)
subset_o['aux']=np.random.binomial(1,.192,o_n)

subset_o['trip_category'] = np.where(subset_o['aux'] == 1, "round-trip", "one-way")
#Borramos la columa auxiliar
del subset_o['aux']
subset_o.head()

In [ ]:
#Juntamos los dos dataframes
data = subset_s.append(subset_o, ignore_index=True)
data.shape

In [ ]:
#Convertimos la duración a minutos
data['duration'] = round(data['tripduration']/60)

In [ ]:
data['starttime'] = pd.to_datetime(data['starttime']).dt.strftime('%Y-%m-%d %H:%M')
data['stoptime'] = pd.to_datetime(data['stoptime']).dt.strftime('%Y-%m-%d %H:%M')

In [ ]:
data_ = data.copy()

In [ ]:
# remove spaces in column names
data_.columns = data_.columns.str.replace(' ', '_')

In [ ]:
# Change to lower categorical variables
cambiar_minusculas_variable(data_,'start_station_name')
cambiar_minusculas_variable(data_,'end_station_name')
cambiar_minusculas_variable(data_,'usertype')
cambiar_minusculas_variable(data_,'type_pass')
cambiar_minusculas_variable(data_,'trip_category')

In [ ]:
# Converting start and end times to datetime objects.
data_['starttime'] = pd.to_datetime(data_['starttime'])
data_['stoptime'] = pd.to_datetime(data_['stoptime'])

In [ ]:
# Create label for rides that were charged based on current pricing plan.
def label_charged(duration):
    """"Function to create a new catergorical variable that indicate if theres has to be an adittional charge"""
    if duration <= 30:
        return 0
    else:
        return 1

data_['charged_ride'] = data_['duration'].apply(label_charged)

In [ ]:
# Column for hour of day of ride.
data_['start_hour'] = data_['starttime'].dt.hour

In [ ]:
latest_ride_date = data_['stoptime'].max()
latest_ride_date

In [ ]:
bike_last_used_id = bike_last_used = data_.groupby('bikeid')['stoptime'].max().keys().tolist()
bike_last_used_date = data_.groupby('bikeid')['stoptime'].max().tolist()

In [ ]:
# Number of bikes active each month in the past year.
for month in range(1,13):
    print(month, len([record for record in list(zip(bike_last_used_id,bike_last_used_date)) if 
                      record[1] > latest_ride_date + np.timedelta64(-month, 'M') ]))

In [ ]:
# Number of inactive bikes (bikes that haven't been used in the past 6 months)
len([record for record in list(zip(bike_last_used_id,bike_last_used_date)) if 
     record[1] < latest_ride_date + np.timedelta64(-6, 'M') ])

In [ ]:
# Create feature indicating active bikes, used in past 6 months. 1 if active, 0 if inactive.
inactive_ids = [record[0] for record in list(zip(bike_last_used_id,bike_last_used_date)) if 
                record[1] < latest_ride_date + np.timedelta64(-6, 'M') ]
data_['active_6m'] = data_['bikeid'].apply(lambda bike_id: 0 if bike_id in (inactive_ids) else 1)

In [ ]:
#Select variables to modeling
data_ = data_ [["trip_id", "duration","starttime", "stoptime", "trip_category", "type_pass"]]